## 1. split the dataset

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

data_dir = '/mntnfs/med_data5/lijingquan/puppy/dataset'  
train_dir = '/mntnfs/med_data5/lijingquan/puppy/train' 
test_dir = '/mntnfs/med_data5/lijingquan/puppy/test'  

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for subdir in ['adult', 'puppy']:
    img_dir = os.path.join(data_dir, subdir)
    img_paths = [os.path.join(img_dir, fname) for fname in os.listdir(img_dir)]
    
    train_paths, test_paths = train_test_split(img_paths, test_size=0.2, random_state=42)
    train_subdir = os.path.join(train_dir, subdir)
    test_subdir = os.path.join(test_dir, subdir)
    
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)
    
    for path in train_paths:
        shutil.copy(path, train_subdir)
    
    for path in test_paths:
        shutil.copy(path, test_subdir)

print("Successfully split the dataset")

ModuleNotFoundError: No module named 'sklearn'

## 2. data loading and pre-processing

In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 设置数据目录
train_dir = '/mntnfs/med_data5/lijingquan/puppy/train'  # 替换为你的训练集路径
test_dir = '/mntnfs/med_data5/lijingquan/puppy/test'  # 替换为你的测试集路径

# 定义数据增强和预处理过程
transform = transforms.Compose([
    transforms.Resize(256),  # 调整图像大小
    transforms.CenterCrop(224),  # 中心裁剪图像到 224x224
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.ToTensor(),  # 转换为 Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])

# 加载训练集和测试集
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# 创建 DataLoader（批量加载数据）
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 查看一个批次的数据
data_iter = iter(train_loader)
images, labels = next(data_iter)  # 使用内置的 next() 函数来获取批次数据
print(images.shape, labels.shape)  # 打印批次中的图像和标签的形状

torch.Size([32, 3, 224, 224]) torch.Size([32])


## 3. build model

In [3]:
import torch
import torch.nn as nn
from torchvision import models

# 加载预训练的 VGG16 模型
model = models.vgg16(pretrained=True)

# 冻结预训练模型的卷积层
for param in model.parameters():
    param.requires_grad = False

# 获取最后一层的输入特征数
num_ftrs = model.classifier[6].in_features

# 替换最后一层（分类层），输出 1 个神经元，用于二分类（狗的年龄）
model.classifier[6] = nn.Linear(num_ftrs, 1)

# 将模型移动到 GPU（如果有的话）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 打印模型结构以查看修改后的网络
print(model)

/home/lijingquan/miniconda3/envs/puppy/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lijingquan/miniconda3/envs/puppy/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

/home/lijingquan/miniconda3/envs/puppy/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647378361/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
import torch.optim as optim
import torch.nn as nn

# 设置损失函数
criterion = nn.BCEWithLogitsLoss()

# 设置优化器，仅训练最后的全连接层
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.0001)

# 将损失函数和优化器移到 GPU（如果有的话）
criterion = criterion.to(device)

## 4. train model

In [ ]:
# 训练模型的函数
from tqdm import tqdm

def train_model(model, criterion, optimizer, train_loader, num_epochs=5):
    model.train()  # 设置为训练模式
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(train_loader, total=len(train_loader), leave=True)
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  # 清空上一步的梯度
            
            # 前向传播
            outputs = model(inputs)  # 输出预测结果
            loss = criterion(outputs.squeeze(), labels.float())  # 计算损失
            
            # 反向传播和优化
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()  # 累加损失
            predicted = (torch.sigmoid(outputs) > 0.5).float()  # 将输出通过sigmoid函数转换为0或1
            # print(f'predicted / labels : {predicted.shape} / {labels.shape}')
            correct += (predicted.view(-1) == labels).sum().item()  # 计算正确预测的个数
            # print(f'\tpredicted.view: {predicted.view(-1).shape, predicted.view(-1)}')
            # print(f'\tlabels: {labels.shape, labels}')
            total += labels.size(0)  # 批次大小
            # print(f'\t> correct / total incrementation :\t{(predicted.view(-1) == labels).sum().item()} / {labels.size(0)}')
        
        epoch_loss = running_loss / len(train_loader)  # 计算平均损失
        epoch_acc = (correct / total) * 100  # 计算准确率并转化为百分比
        print(f'correct / total : {correct} / {total}')
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
    
    return model

# 训练模型，训练5个epoch
train_model(model, criterion, optimizer, train_loader, num_epochs=5)

100%|██████████| 25/25 [01:37<00:00,  3.89s/it]


correct / total : 587 / 800
Epoch 1/5, Loss: 0.5150, Accuracy: 73.38%


100%|██████████| 25/25 [01:35<00:00,  3.83s/it]


correct / total : 615 / 800
Epoch 2/5, Loss: 0.4949, Accuracy: 76.88%


100%|██████████| 25/25 [01:34<00:00,  3.79s/it]


correct / total : 609 / 800
Epoch 3/5, Loss: 0.4970, Accuracy: 76.12%


100%|██████████| 25/25 [01:34<00:00,  3.79s/it]


correct / total : 599 / 800
Epoch 4/5, Loss: 0.5059, Accuracy: 74.88%


 12%|█▏        | 3/25 [00:10<01:16,  3.48s/it]